In [1]:
from sklearn import preprocessing
from sklearn.preprocessing import MinMaxScaler, RobustScaler, StandardScaler
from sklearn.linear_model import LinearRegression, LassoCV
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, mean_squared_error
from sklearn.preprocessing import PowerTransformer
import math
import warnings
warnings.filterwarnings('ignore')
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
from scipy import stats
import matplotlib.style as style
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from scipy.stats import norm, skew #for some statistics
from sklearn.preprocessing import PowerTransformer
from sklearn.model_selection import GridSearchCV
# 處理 data 套件
import numpy as np
import pandas as pd
from xgboost.sklearn import XGBRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score,KFold
from scipy.stats import skew

In [2]:
def skew_pro(data):
    columns=data.drop(['Predict'], axis=1).columns
    for col in columns: 
        if abs(data[col].skew()) >= 0.7: 
            pt = PowerTransformer() 
            d = pt.fit_transform(data[col].values.reshape(-1, 1)).flatten()
            data[col]=d
    X = data[columns]
    scaler = RobustScaler()
    data[columns] = scaler.fit_transform(X)
    
    return data

def lr_rmse_ave(x,y):
    
    
    train_rmse=[]
    test_rmse=[]
    test_r2=[]
    
    for i in np.arange(10):
        x_train,x_test,y_train,y_test = train_test_split(x,y, random_state = i)
        lr = LinearRegression().fit(x_train,y_train)

        y_train_pred = lr.predict(x_train)
        y_test_pred = lr.predict(x_test)
        train_rmse.append(mean_squared_error(y_train, y_train_pred,squared=False))
        test_rmse.append(mean_squared_error(y_test, y_test_pred,squared=False))
        test_r2.append(r2_score(y_test,y_test_pred))
    
    train_rmse=np.array(train_rmse).mean()
    test_rmse=np.array(test_rmse).mean()
    test_r2=np.array(test_r2).mean()
    
#     print('train_rmse:', train_rmse)
#     print('test_rmse:', test_rmse)
#     print('test_r2:', test_r2)
    return train_rmse, test_rmse, test_r2
    

In [3]:
#
n='Input_A1_020'  #改路徑
#
dire= ['./BayesianRidge_Pre_0/','./BayesianRidge_Pre_1/','./BayesianRidge_Pre_2/','./BayesianRidge_Pre_3/','./BayesianRidge_Pre_4/','./BayesianRidge_Pre_5/','./BayesianRidge_Pre_6/','./BayesianRidge_Pre_7/']
# dire= ['../BayesianRidge_Pre_0/']
print(n)
for d in dire:
    f_train_rmse=[]
    f_test_rmse=[]
    f_test_r2=[]
    print(d)
    data_d= d+n+'.csv'
    data= pd.read_csv(data_d,index_col=0).drop(['Number'], axis=1)
    y=data.Predict
    skew_data=skew_pro(data)
    num=skew_data.select_dtypes(exclude='object')
    numcorr=num.corr()
    f20=abs(numcorr['Predict']).sort_values(ascending=False).head(21).to_frame().index.to_numpy()[1:]
    fea_list=[2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20]
    for i in fea_list:
        print('feature number:',i)
        x=skew_data[f20[:i]]
        tr_r,te_r, te_r2= lr_rmse_ave(x,y)
        print('train_rmse:',tr_r)
        print('test_rmse:',te_r)
        print('test_r2:',te_r2)
        f_train_rmse.append(tr_r)
        f_test_rmse.append(te_r)
        f_test_r2.append(te_r2)

    test_r=np.array(f_test_rmse)
    test_r2=np.array(f_test_r2)
    test_r_min= np.where(test_r == np.amin(test_r))
    test_r2_max= np.where(test_r2 == np.amax(test_r2))    
    print('Test_rmse_min = 1.fea_num:', fea_list[test_r_min[0][0]], ' 2.rmse:' ,np.amin(test_r))
    print('Test_r2_max = 1. fea_num:', fea_list[test_r2_max[0][0]],  ' 2.r2:', np.amax(test_r2))



Input_A1_020
./BayesianRidge_Pre_0/
feature number: 2
train_rmse: 0.5928217486273215
test_rmse: 0.646585647097155
test_r2: 0.370698679108309
feature number: 3
train_rmse: 0.5844720276614522
test_rmse: 0.6511332225562532
test_r2: 0.36485135531258117
feature number: 4
train_rmse: 0.5831209556536728
test_rmse: 0.6568360159671985
test_r2: 0.3545339642275792
feature number: 5
train_rmse: 0.5824025653306407
test_rmse: 0.6594662460327358
test_r2: 0.3493447817400817
feature number: 6
train_rmse: 0.5820069840234381
test_rmse: 0.659608719896304
test_r2: 0.34914455883802875
feature number: 7
train_rmse: 0.5817708592190032
test_rmse: 0.6596553345534402
test_r2: 0.3489464324693693
feature number: 8
train_rmse: 0.5807490080212043
test_rmse: 0.6589953026259924
test_r2: 0.3504795277327843
feature number: 9
train_rmse: 0.5740197085483204
test_rmse: 0.6482825019788352
test_r2: 0.3703332570152188
feature number: 10
train_rmse: 0.5683738583407798
test_rmse: 0.6415633131733706
test_r2: 0.38259426691083415


train_rmse: 0.5463074682467943
test_rmse: 0.6409081063338103
test_r2: 0.38229308352614166
feature number: 19
train_rmse: 0.5459588158279103
test_rmse: 0.6423494494293135
test_r2: 0.379465546860774
feature number: 20
train_rmse: 0.5419630951882063
test_rmse: 0.6448431567128531
test_r2: 0.3740035541492949
Test_rmse_min = 1.fea_num: 14  2.rmse: 0.6306883477951887
Test_r2_max = 1. fea_num: 14  2.r2: 0.40208731625274047
./BayesianRidge_Pre_4/
feature number: 2
train_rmse: 0.5928217486273215
test_rmse: 0.646585647097155
test_r2: 0.370698679108309
feature number: 3
train_rmse: 0.5844720276614522
test_rmse: 0.6511332225562532
test_r2: 0.36485135531258117
feature number: 4
train_rmse: 0.5831209556536728
test_rmse: 0.6568360159671985
test_r2: 0.3545339642275792
feature number: 5
train_rmse: 0.5824025653306407
test_rmse: 0.6594662460327358
test_r2: 0.3493447817400817
feature number: 6
train_rmse: 0.5820069840234381
test_rmse: 0.659608719896304
test_r2: 0.34914455883802875
feature number: 7
train_

train_rmse: 0.5489977944588103
test_rmse: 0.6309090424928409
test_r2: 0.40156610938717535
feature number: 16
train_rmse: 0.548011847249071
test_rmse: 0.6315559262749826
test_r2: 0.4001704584213247
feature number: 17
train_rmse: 0.5472923990757492
test_rmse: 0.6345618407486624
test_r2: 0.3943362121363597
feature number: 18
train_rmse: 0.5462939948416475
test_rmse: 0.6408738846676534
test_r2: 0.38227826281337285
feature number: 19
train_rmse: 0.5459433960989357
test_rmse: 0.6423011087612187
test_r2: 0.3794804581881894
feature number: 20
train_rmse: 0.5419598303390509
test_rmse: 0.6447831681643836
test_r2: 0.3740598104014512
Test_rmse_min = 1.fea_num: 14  2.rmse: 0.6306442436576548
Test_r2_max = 1. fea_num: 14  2.r2: 0.40211777854732256


In [4]:
fea_number=14
data=pd.read_csv('./BayesianRidge_Pre_0/Input_A1_020.csv',index_col=0).drop(['Number'], axis=1)
y=data.Predict
skew_data=skew_pro(data)
num=skew_data.select_dtypes(exclude='object')
numcorr=num.corr()
title=abs(numcorr['Predict']).sort_values(ascending=False).head(fea_number+1).to_frame().index.to_numpy()[1:]
title

array(['Input_A4_020', 'Input_A3_020', 'Input_A5_020', 'Input_A6_020',
       'Input_A2_020', 'Input_A1_021', 'Input_A2_021', 'Input_A2_015',
       'Input_C_001', 'Input_A1_016', 'Input_C_060', 'Input_A1_019',
       'Input_A4_011', 'Input_A2_019'], dtype=object)

## Linear Regression Baseline

In [5]:
def lr_rmse_ave_fea(data,fea_num):
    
    num=data.select_dtypes(exclude='object')
    numcorr=num.corr()
    cols=abs(numcorr['Predict']).sort_values(ascending=False).head(fea_num+1).to_frame().index.to_numpy()[1:]
    y = data['Predict']
    X= data[cols]
    train_rmse=[]
    test_rmse=[]
    test_r2=[]
    
    for i in np.arange(10):
        x_train,x_test,y_train,y_test = train_test_split(X,y, random_state = i)
        lr = LinearRegression().fit(x_train,y_train)

        y_train_pred = lr.predict(x_train)
        y_test_pred = lr.predict(x_test)
        train_rmse.append(mean_squared_error(y_train, y_train_pred,squared=False))
        test_rmse.append(mean_squared_error(y_test, y_test_pred,squared=False))
        test_r2.append(r2_score(y_test,y_test_pred))
    
    train_rmse=np.array(train_rmse).mean()
    test_rmse=np.array(test_rmse).mean()
    test_r2=np.array(test_r2).mean()
    
    print('train_rmse:', train_rmse)
    print('test_rmse:', test_rmse)
    print('test_r2:', test_r2)
#     return train_rmse, test_rmse, test_r2

In [6]:
lr_rmse_ave_fea(skew_data,14)

train_rmse: 0.5496703758835424
test_rmse: 0.630685153741493
test_r2: 0.4021163900449416


## Lasso

In [7]:
def laso_alpha(x,y,parameters):
    from sklearn.linear_model import Lasso
    
    para=[]

    for i in np.arange(10):
        X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=i)
        lasso=Lasso()
        lasso_reg=GridSearchCV(lasso, param_grid=parameters, scoring='neg_mean_squared_error', cv=3)
        lasso_reg.fit(X_train,y_train)
        para.append(lasso_reg.best_params_)

    print(para)
    
def laso_rmse_ave(x,y,alp):
    from sklearn.linear_model import Lasso
    
    rmse=[]
    r2=[]

    for i in np.arange(10):
        X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=i)
        lasso_mod=Lasso(alpha=alp)
        lasso_mod.fit(X_train,y_train)
        y_lasso_train=lasso_mod.predict(X_train)
        y_lasso_test=lasso_mod.predict(X_test)
        rmse.append(math.sqrt(mean_squared_error(y_test, y_lasso_test)))
        r2.append(r2_score(y_test, y_lasso_test))
    
    test_rmse=np.array(rmse).mean()
    print('test_rmse_ave:',test_rmse)
    print(rmse)
    print('\n')
    test_r2=np.array(r2).mean()
    print('test_r2_ave:',test_r2)
    print(r2)
    

In [42]:
parameters= {'alpha':[0.009,0.01,0.02,0.005]}
laso_alpha(skew_data[title],y,parameters)

[{'alpha': 0.01}, {'alpha': 0.01}, {'alpha': 0.01}, {'alpha': 0.01}, {'alpha': 0.01}, {'alpha': 0.009}, {'alpha': 0.01}, {'alpha': 0.02}, {'alpha': 0.02}, {'alpha': 0.02}]


In [8]:
laso_rmse_ave(skew_data[title],y,0.01)

test_rmse_ave: 0.5966543862816749
[0.46620984911224955, 0.6741264570147933, 0.8203464865214706, 0.4369860546979481, 0.6401251047894848, 0.7456655545103595, 0.7047084919188028, 0.6229993612177261, 0.4866207589069143, 0.36875574412700046]


test_r2_ave: 0.4172470972694066
[0.5921123062469382, 0.33779988139965444, 0.41364601517656396, 0.4194992900160419, 0.5331287816228147, 0.23563419886713255, 0.32234871859884107, 0.3085633111767526, 0.4348570496571701, 0.5748814199321561]


## ElasticNet

In [9]:
def score(y_pred):
    return str(math.sqrt(mean_squared_error(y_test, y_pred)))

def ElasticNet_alpha(x,y,alphas,l1ratio):
    from sklearn.linear_model import ElasticNetCV
    
    alpha_=[]
    l1_ratio_=[]

    for i in np.arange(10):
        X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=i)
        elastic_cv = ElasticNetCV(cv=3, max_iter=1e7, alphas=alphas,  l1_ratio=l1ratio)
        elasticmod = elastic_cv.fit(X_train, y_train.ravel())

        alpha_.append(elastic_cv.alpha_)
        l1_ratio_.append(elastic_cv.l1_ratio_)

    print('alpha')
    print(alpha_)
    print('\n')
    print('l1_ratio')
    print(l1_ratio_)
    
def ElasticNet_rmse_ave(x,y,alp,l1r):
    from sklearn.linear_model import ElasticNet
    
    rmse=[]
    r2=[]

    for i in np.arange(10):
        X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=i)
        ElasticNet_mod=ElasticNet(alpha=alp, l1_ratio=l1r)
        ElasticNet_mod.fit(X_train,y_train)
        y_ElasticNet_train=ElasticNet_mod.predict(X_train)
        y_ElasticNet_test=ElasticNet_mod.predict(X_test)
        rmse.append(math.sqrt(mean_squared_error(y_test, y_ElasticNet_test)))
        r2.append(r2_score(y_test, y_ElasticNet_test))
    
    test_rmse=np.array(rmse).mean()
    print('test_rmse_ave:',test_rmse)
    print(rmse)
    print('\n')
    test_r2=np.array(r2).mean()
    print('test_r2_ave:',test_r2)
    print(r2)
    

In [49]:
# alphas = [10,1,0.1,0.01,0.001,0.002,0.003,0.004,0.005,0.00054255]
alphas =[0.09]
# alphas =[0.07,0.06,0.05,0.04, 0.03,0.02]
l1ratio = [0.02,0.03,0.04,0.05,0.08,]
ElasticNet_alpha(skew_data[title],y,alphas,l1ratio)

alpha
[0.09, 0.09, 0.09, 0.09, 0.09, 0.09, 0.09, 0.09, 0.09, 0.09]


l1_ratio
[0.02, 0.04, 0.02, 0.02, 0.02, 0.02, 0.02, 0.08, 0.03, 0.05]


In [10]:
ElasticNet_rmse_ave(skew_data[title],y,[0.09],0.02)

test_rmse_ave: 0.5899816492677211
[0.471493396911073, 0.6503581355816648, 0.8399015358747844, 0.4255020295801734, 0.643091779783245, 0.7324902145547428, 0.714958298947838, 0.5941570359776989, 0.478868904597491, 0.34899516086850063]


test_r2_ave: 0.4338190480252001
[0.5828147508562611, 0.3836723310827619, 0.38535834861637475, 0.4496095730017958, 0.5287913022957005, 0.2624070804541806, 0.30249282821691914, 0.3711027346564324, 0.4527190577421931, 0.6192224733293814]


## SVR

In [11]:
from sklearn.svm import SVR

In [61]:
# parameters = {'kernel': ('linear', 'rbf','poly'), 'C':[1.2,1.3,1,1.5],'gamma': [1e-7,1e-6 1e-8,5e-7],'epsilon':[0.2,0.5,0.3,0.4]}
for i in np.arange(10):
    X_SVR_train, X_SVR_test, y_SVR_train, y_SVR_test = train_test_split(skew_data[title], y, test_size=0.2, random_state=i)
    parameters = {'kernel': ['linear'], 'C':[1.2,1.3,1,1.4,1.5],'gamma': [1e-7,],'epsilon':[0.4]}
    clf = GridSearchCV(estimator = SVR(), param_grid = parameters , n_jobs=6,iid=False, verbose=0,scoring='neg_mean_squared_error')
    clf.fit(X_SVR_train,y_SVR_train)
    print('best params')
    print (clf.best_params_)
    print('\n')

best params
{'C': 1.2, 'epsilon': 0.4, 'gamma': 1e-07, 'kernel': 'linear'}


best params
{'C': 1, 'epsilon': 0.4, 'gamma': 1e-07, 'kernel': 'linear'}


best params
{'C': 1.4, 'epsilon': 0.4, 'gamma': 1e-07, 'kernel': 'linear'}


best params
{'C': 1.4, 'epsilon': 0.4, 'gamma': 1e-07, 'kernel': 'linear'}


best params
{'C': 1, 'epsilon': 0.4, 'gamma': 1e-07, 'kernel': 'linear'}


best params
{'C': 1.2, 'epsilon': 0.4, 'gamma': 1e-07, 'kernel': 'linear'}


best params
{'C': 1.4, 'epsilon': 0.4, 'gamma': 1e-07, 'kernel': 'linear'}


best params
{'C': 1.5, 'epsilon': 0.4, 'gamma': 1e-07, 'kernel': 'linear'}


best params
{'C': 1, 'epsilon': 0.4, 'gamma': 1e-07, 'kernel': 'linear'}


best params
{'C': 1.5, 'epsilon': 0.4, 'gamma': 1e-07, 'kernel': 'linear'}




In [12]:
#best params {'C': 1.3, 'epsilon': 0.4, 'gamma': 1e-07, 'kernel': 'linear'}
def svr_rmse_ave(data,fea_num,model):
    
    num=data.select_dtypes(exclude='object')
    numcorr=num.corr()
    title=abs(numcorr['Predict']).sort_values(ascending=False).head(fea_num+1).to_frame().index.to_numpy()[1:]
    y = data['Predict']
    X= data[title]
    train_rmse=[]
    test_rmse=[]
    test_r2=[]
    
    for i in np.arange(10):
        x_train,x_test,y_train,y_test = train_test_split(X,y, random_state = i)
        svr = model.fit(x_train,y_train)
        y_train_pred = svr.predict(x_train)
        y_test_pred = svr.predict(x_test)
        train_rmse.append(mean_squared_error(y_train, y_train_pred,squared=False))
        test_rmse.append(mean_squared_error(y_test, y_test_pred,squared=False))
        test_r2.append(r2_score(y_test,y_test_pred))
    
    train_rmse=np.array(train_rmse).mean()
    test_rmse=np.array(test_rmse).mean()
    test_r2=np.array(test_r2).mean()
    
    print('train_rmse:', train_rmse)
    print('test_rmse:', test_rmse)
    print('test_r2:', test_r2)
#     return train_rmse, test_rmse, test_r2

In [13]:
model=SVR(kernel='linear', C=1.3, gamma= 1e-07, epsilon= 0.4)
svr_rmse_ave(skew_data,14,model)

train_rmse: 0.5648260361049923
test_rmse: 0.6284314864841181
test_r2: 0.409871488238135


## XGB

In [14]:
def xgb_ave(x,y,model):
    
    rmse=[]
    r2=[]

    for i in np.arange(10):
        X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=i)
        model.fit(X_train,y_train)
        y_pred = model.predict(X_test)
        rmse.append(math.sqrt(mean_squared_error(y_test, y_pred )))
        r2.append(r2_score(y_test, y_pred ))
    
    test_rmse=np.array(rmse).mean()
    print('test_rmse_ave:',test_rmse)
    print(rmse)
    print('\n')
    test_r2=np.array(r2).mean()
    print('test_r2_ave:',test_r2)
    print(r2)
    

### 先調 booster / n_estimators

In [71]:
#for tuning parameters
parameters_for_testing = {
#     'booster':['gbtree','gblinear','dart'],
    'booster':['dart'],
    'n_estimators':range(10,201,10),
}

other_params = {'learning_rate': 0.1, 'n_estimators': 90, 'max_depth': 5, 'min_child_weight': 1, 'seed': 42,
'subsample': 0.8, 'colsample_bytree': 0.8, 'gamma': 0, 'reg_alpha': 0, 'reg_lambda': 1}

for i in np.arange(10):
    X_XGB_train, X_XGB_test, y_XGB_train, y_XGB_test = train_test_split(skew_data[title], y, test_size=0.2, random_state=i)
    xgb_model = XGBRegressor(**other_params)
    gsearch = GridSearchCV(estimator = xgb_model, param_grid = parameters_for_testing, n_jobs=6,iid=False, verbose=0,scoring='neg_mean_squared_error')
    gsearch.fit(X_XGB_train,y_XGB_train)
    print('best params')
    print (gsearch.best_params_)
    print('\n')


[02:47:48] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
best params
{'booster': 'dart', 'n_estimators': 20}


[02:47:57] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
best params
{'booster': 'dart', 'n_estimators': 20}


[02:48:04] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
best params
{'booster': 'dart', 'n_estimators': 30}


[02:48:12] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
best params
{'booster': 'dart', 'n_estimators': 20}


[02:48:19] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
best params
{'booster': 'dart', 'n_estimators': 20}


[02:48:26] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
best params
{'booster': 'dart', 'n

### 調 max_depth / min_child_weight

In [73]:
# {'booster': 'dart', 'n_estimators': 30}

parameters_for_testing = {
#     'max_depth': [1,2,3, 4, 5, 6, 7, 8, 9, 10],
    'max_depth': [5], 
    'min_child_weight': [1, 2, 3, 4, 5, 6]
}

other_params = {'booster':'dart','learning_rate': 0.1, 'n_estimators': 30, 'seed': 42,
'subsample': 0.8, 'colsample_bytree': 0.8, 'gamma': 0, 'reg_alpha': 0, 'reg_lambda': 1}

for i in np.arange(10):
    X_XGB_train, X_XGB_test, y_XGB_train, y_XGB_test = train_test_split(skew_data[title], y, test_size=0.2, random_state=i)
    xgb_model = XGBRegressor(**other_params)
    gsearch = GridSearchCV(estimator = xgb_model, param_grid = parameters_for_testing, n_jobs=6,iid=False, verbose=0,scoring='neg_mean_squared_error')
    gsearch.fit(X_XGB_train,y_XGB_train)
    print('best params')
    print (gsearch.best_params_)
    print('\n')

[02:53:39] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
best params
{'max_depth': 5, 'min_child_weight': 4}


[02:53:40] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
best params
{'max_depth': 5, 'min_child_weight': 6}


[02:53:41] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
best params
{'max_depth': 5, 'min_child_weight': 5}


[02:53:41] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
best params
{'max_depth': 5, 'min_child_weight': 3}


[02:53:42] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
best params
{'max_depth': 5, 'min_child_weight': 1}


[02:53:42] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
best params
{'max_depth': 5, 'min_

### 調 gamma

In [74]:
# {'max_depth': 5, 'min_child_weight': 4}
parameters_for_testing = {
    'gamma': [0,0.1, 0.2, 0.3, 0.4, 0.5, 0.6]
}

other_params = {'booster':'dart','learning_rate': 0.1, 'n_estimators': 30, 'max_depth': 5, 'min_child_weight': 4, 'seed': 42,
'subsample': 0.8, 'colsample_bytree': 0.8, 'reg_alpha': 0, 'reg_lambda': 1}

for i in np.arange(10):
    X_XGB_train, X_XGB_test, y_XGB_train, y_XGB_test = train_test_split(skew_data[title], y, test_size=0.2, random_state=i)
    xgb_model = XGBRegressor(**other_params)
    gsearch = GridSearchCV(estimator = xgb_model, param_grid = parameters_for_testing, n_jobs=6,iid=False, verbose=0,scoring='neg_mean_squared_error')
    gsearch.fit(X_XGB_train,y_XGB_train)
    print('best params')
    print (gsearch.best_params_)
    print('\n')

[02:54:38] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
best params
{'gamma': 0}


[02:54:39] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
best params
{'gamma': 0.4}


[02:54:39] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
best params
{'gamma': 0.3}


[02:54:40] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
best params
{'gamma': 0}


[02:54:41] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
best params
{'gamma': 0.3}


[02:54:42] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
best params
{'gamma': 0.1}


[02:54:43] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
best params
{'gamm

### 調 subsample / colsample_bytree

In [78]:
# {'gamma': 0.2}
parameters_for_testing = {
#     'colsample_bytree': [0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9],
    'colsample_bytree': [0.5],
    'subsample': [0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9] 
}

other_params = {'booster':'dart','learning_rate': 0.1, 'n_estimators': 30, 'max_depth': 5, 'min_child_weight': 4, 'seed': 42,
 'reg_alpha': 0, 'reg_lambda': 1, 'gamma':0.2}

for i in np.arange(10):
    X_XGB_train, X_XGB_test, y_XGB_train, y_XGB_test = train_test_split(skew_data[title], y, test_size=0.2, random_state=i)
    xgb_model = XGBRegressor(**other_params)
    gsearch = GridSearchCV(estimator = xgb_model, param_grid = parameters_for_testing, n_jobs=6,iid=False, verbose=0,scoring='neg_mean_squared_error')
    gsearch.fit(X_XGB_train,y_XGB_train)
    print('best params')
    print (gsearch.best_params_)
    print('\n')

[02:59:43] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
best params
{'colsample_bytree': 0.5, 'subsample': 0.2}


[02:59:44] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
best params
{'colsample_bytree': 0.5, 'subsample': 0.2}


[02:59:44] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
best params
{'colsample_bytree': 0.5, 'subsample': 0.5}


[02:59:45] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
best params
{'colsample_bytree': 0.5, 'subsample': 0.8}


[02:59:46] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
best params
{'colsample_bytree': 0.5, 'subsample': 0.2}


[02:59:46] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
best params
{'

### reg_alpha / reg_lambda

In [80]:
# {'colsample_bytree': 0.5, 'subsample': 0.4}

parameters_for_testing = {
#     'reg_alpha': [0,0.05, 0.1, 0.5 , 1, 2, 3], 
    'reg_alpha': [0.5],     
    'reg_lambda': [0.05, 0.1, 0.5 , 1, 2, 3]
}

other_params = {'booster':'dart','learning_rate': 0.1, 'n_estimators': 30, 'max_depth': 5, 'min_child_weight': 4, 'seed': 42,
 'gamma':0.2,'subsample':0.4,'colsample_bytree': 0.5}


for i in np.arange(10):
    X_XGB_train, X_XGB_test, y_XGB_train, y_XGB_test = train_test_split(skew_data[title], y, test_size=0.2, random_state=i)
    xgb_model = XGBRegressor(**other_params)
    gsearch = GridSearchCV(estimator = xgb_model, param_grid = parameters_for_testing, n_jobs=6,iid=False, verbose=0,scoring='neg_mean_squared_error')
    gsearch.fit(X_XGB_train,y_XGB_train)
    print('best params')
    print (gsearch.best_params_)
    print('\n')

[03:03:07] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
best params
{'reg_alpha': 0.5, 'reg_lambda': 3}


[03:03:07] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
best params
{'reg_alpha': 0.5, 'reg_lambda': 3}


[03:03:08] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
best params
{'reg_alpha': 0.5, 'reg_lambda': 0.05}


[03:03:08] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
best params
{'reg_alpha': 0.5, 'reg_lambda': 3}


[03:03:09] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
best params
{'reg_alpha': 0.5, 'reg_lambda': 2}


[03:03:09] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
best params
{'reg_alpha': 0.5, 'reg_lambda': 0.1}



### learning rate

In [81]:
# {'reg_alpha': 0.5, 'reg_lambda': 1.5}

parameters_for_testing = {
   'learning_rate': [0.01, 0.05, 0.07, 0.1, 0.2]
}

other_params = {'booster':'dart', 'n_estimators': 30, 'max_depth': 5, 'min_child_weight': 4, 'seed': 42,
 'gamma':0.2,'subsample':0.4,'colsample_bytree': 0.5,'reg_alpha': 0.5, 'reg_lambda': 1.5}

for i in np.arange(10):
    X_XGB_train, X_XGB_test, y_XGB_train, y_XGB_test = train_test_split(skew_data[title], y, test_size=0.2, random_state=i)
    xgb_model = XGBRegressor(**other_params)
    gsearch = GridSearchCV(estimator = xgb_model, param_grid = parameters_for_testing, n_jobs=6,iid=False, verbose=0,scoring='neg_mean_squared_error')
    gsearch.fit(X_XGB_train,y_XGB_train)
    print('best params')
    print (gsearch.best_params_)
    print('\n')

[03:05:29] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
best params
{'learning_rate': 0.07}


[03:05:29] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
best params
{'learning_rate': 0.1}


[03:05:30] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
best params
{'learning_rate': 0.1}


[03:05:30] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
best params
{'learning_rate': 0.1}


[03:05:30] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
best params
{'learning_rate': 0.1}


[03:05:31] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
best params
{'learning_rate': 0.1}


[03:05:31] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprec

In [15]:
best_xgb_model = XGBRegressor(
                objective ='reg:squarederror',
              learning_rate = 0.09,
              booster = 'gbtree', 
              n_estimators = 30, 
              max_depth = 5, 
              min_child_weight = 4,
              seed = 42,
              gamma = 0.2,
              subsample = 0.4,
              colsample_bytree = 0.5,
              reg_alpha =  0.5,
              reg_lambda = 1.5)
xgb_ave(skew_data[title], y,best_xgb_model)

test_rmse_ave: 0.5538214827832086
[0.415311279394196, 0.5479720012596659, 0.8479936716277775, 0.39496177295789897, 0.6421760293128854, 0.6499519208695562, 0.6900304877959511, 0.6119958926029, 0.38985330346548364, 0.3479684685457719]


test_r2_ave: 0.5029196661832124
[0.6763130676748439, 0.562454473777501, 0.3734576111103507, 0.5257823356859532, 0.5301323312728741, 0.41926828050926446, 0.3502836235533211, 0.3327720472170135, 0.6372733304228446, 0.621459560608157]


## RandomForest

In [16]:
def rand_ave(x,y,model):
    
    rmse=[]
    r2=[]

    for i in np.arange(10):
        X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=i)
        model.fit(X_train,y_train)
        y_pred = model.predict(X_test)
        rmse.append(math.sqrt(mean_squared_error(y_test, y_pred )))
        r2.append(r2_score(y_test, y_pred ))
    
    test_rmse=np.array(rmse).mean()
    print('test_rmse_ave:',test_rmse)
    print(rmse)
    print('\n')
    test_r2=np.array(r2).mean()
    print('test_r2_ave:',test_r2)
    print(r2)
    

In [87]:
from sklearn.ensemble import RandomForestRegressor
clf = RandomForestRegressor(
          criterion ='mse', 
          random_state = 42
)

In [88]:
X_RAND_train, X_RAND_test, y_RAND_train, y_RAND_test = train_test_split(skew_data[title], y, test_size=0.2, random_state=42)

In [92]:
param_grid = {
              'n_estimators': [50,100,200 ,500, 900],
              'max_depth': range(1,10,1),
              'min_samples_split': range(2,10,1)}

gsearch = GridSearchCV(estimator = clf, param_grid = param_grid, n_jobs=6,iid=False, verbose=10,scoring='neg_mean_squared_error')

gsearch.fit(X_RAND_train,y_RAND_train)

print('best params')
print (gsearch.best_params_)
print('best score')
print (gsearch.best_score_)

Fitting 5 folds for each of 360 candidates, totalling 1800 fits


[Parallel(n_jobs=6)]: Using backend LokyBackend with 6 concurrent workers.
[Parallel(n_jobs=6)]: Done   1 tasks      | elapsed:    0.1s
[Parallel(n_jobs=6)]: Done   6 tasks      | elapsed:    0.3s
[Parallel(n_jobs=6)]: Done  13 tasks      | elapsed:    1.3s
[Parallel(n_jobs=6)]: Done  20 tasks      | elapsed:    3.6s
[Parallel(n_jobs=6)]: Done  29 tasks      | elapsed:    7.0s
[Parallel(n_jobs=6)]: Done  38 tasks      | elapsed:    8.5s
[Parallel(n_jobs=6)]: Done  49 tasks      | elapsed:   13.6s
[Parallel(n_jobs=6)]: Done  60 tasks      | elapsed:   16.0s
[Parallel(n_jobs=6)]: Done  73 tasks      | elapsed:   19.4s
[Parallel(n_jobs=6)]: Done  86 tasks      | elapsed:   22.0s
[Parallel(n_jobs=6)]: Done 101 tasks      | elapsed:   26.5s
[Parallel(n_jobs=6)]: Done 116 tasks      | elapsed:   30.1s
[Parallel(n_jobs=6)]: Done 133 tasks      | elapsed:   34.0s
[Parallel(n_jobs=6)]: Done 150 tasks      | elapsed:   39.0s
[Parallel(n_jobs=6)]: Done 169 tasks      | elapsed:   43.2s
[Parallel(

best params
{'max_depth': 2, 'min_samples_split': 7, 'n_estimators': 500}
best score
-0.3918457725449359


In [93]:
# {'max_depth': 2, 'min_samples_split': 7, 'n_estimators': 500} 
param_grid = {
              "min_samples_leaf": range(1,10,1),
              "max_leaf_nodes": range(2,15,1)
              }
    
clf = RandomForestRegressor(
            criterion ='mse', 
            n_estimators = 500,
            max_depth = 2,
            min_samples_split = 7,
            random_state = 42
)

gsearch = GridSearchCV(estimator = clf, param_grid = param_grid, n_jobs=6,iid=False, verbose=10,scoring='neg_mean_squared_error')

gsearch.fit(X_RAND_train,y_RAND_train)

print('best params')
print (gsearch.best_params_)
print('best score')
print (gsearch.best_score_)

Fitting 5 folds for each of 117 candidates, totalling 585 fits


[Parallel(n_jobs=6)]: Using backend LokyBackend with 6 concurrent workers.
[Parallel(n_jobs=6)]: Done   1 tasks      | elapsed:    2.1s
[Parallel(n_jobs=6)]: Done   6 tasks      | elapsed:    2.2s
[Parallel(n_jobs=6)]: Done  13 tasks      | elapsed:    8.9s
[Parallel(n_jobs=6)]: Done  20 tasks      | elapsed:   11.4s
[Parallel(n_jobs=6)]: Done  29 tasks      | elapsed:   13.7s
[Parallel(n_jobs=6)]: Done  38 tasks      | elapsed:   17.9s
[Parallel(n_jobs=6)]: Done  49 tasks      | elapsed:   22.0s
[Parallel(n_jobs=6)]: Done  60 tasks      | elapsed:   24.8s
[Parallel(n_jobs=6)]: Done  73 tasks      | elapsed:   30.4s
[Parallel(n_jobs=6)]: Done  86 tasks      | elapsed:   35.3s
[Parallel(n_jobs=6)]: Done 101 tasks      | elapsed:   39.5s
[Parallel(n_jobs=6)]: Done 116 tasks      | elapsed:   45.2s
[Parallel(n_jobs=6)]: Done 133 tasks      | elapsed:   54.9s
[Parallel(n_jobs=6)]: Done 150 tasks      | elapsed:  1.0min
[Parallel(n_jobs=6)]: Done 169 tasks      | elapsed:  1.2min
[Parallel(

best params
{'max_leaf_nodes': 3, 'min_samples_leaf': 5}
best score
-0.3777524890307898


In [17]:
from sklearn.ensemble import RandomForestRegressor
rand_model = RandomForestRegressor( 
            criterion ='mse', 
            n_estimators = 500,
            max_depth = 2,
            min_samples_split = 7,
            max_leaf_nodes = 3,
            min_samples_leaf =5,
            random_state = 42) 
rand_ave(skew_data[title], y,rand_model)

test_rmse_ave: 0.5992695000065618
[0.42449305712875873, 0.6765634229439021, 0.8571850322298274, 0.46536364368303335, 0.6550714383674187, 0.6951032756058838, 0.7247560260997505, 0.6925960420160501, 0.4417597571020723, 0.359803304888921]


test_r2_ave: 0.41013784632330685
[0.6618425988269516, 0.3330035231537237, 0.35980188048584527, 0.3416565756421902, 0.5110722241981486, 0.3357802960066595, 0.28324468147231285, 0.14545049250183517, 0.5342537657333033, 0.5952724252120978]
